# Infer-2-Gaussian-Mixtures : Distributions Gaussiennes et Melanges

**Serie** : Programmation Probabiliste avec Infer.NET (2/13)  
**Duree estimee** : 50 minutes  
**Prerequis** : Infer-1-Setup

---

## Objectifs

- Maitriser les distributions Gaussienne et Gamma
- Comprendre la notion de priors conjugues
- Implementer l'apprentissage de parametres
- Construire des modeles de melange de Gaussiennes
- Utiliser `Variable.Switch` pour les modeles a composantes

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-1-Setup](Infer-1-Setup.ipynb) | [Infer-3-Factor-Graphs](Infer-3-Factor-Graphs.ipynb) |

---

## 1. Configuration

Comme pour chaque notebook de cette serie, nous commencons par charger les packages Infer.NET et importer les espaces de noms necessaires. Cette etape standardisee garantit que tous les exemples sont reproductibles et que les distributions, algorithmes et outils de modelisation sont disponibles.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


## 2. Scenario : Le Cycliste

### Contexte

Vous vous rendez au travail a velo chaque jour. Votre temps de trajet varie :
- Certains jours, le trajet est rapide (peu de trafic)
- D'autres jours, il est plus lent (embouteillages, meteo)

### Objectifs

1. **Apprendre** la distribution du temps de trajet a partir d'observations
2. **Predire** le temps de trajet de demain
3. **Calculer** des probabilites (ex: P(trajet < 18 min))

### Modelisation

Nous modelerons le temps de trajet avec une distribution **Gaussienne** :

$$\text{temps} \sim \mathcal{N}(\mu, \tau^{-1})$$

- $\mu$ : moyenne (temps moyen de trajet)
- $\tau$ : precision (inverse de la variance)

## 3. Distributions Conjuguees

### Theorie

En inference bayesienne, une distribution **a priori conjuguee** permet une mise a jour analytique simple :

| Vraisemblance | Prior Conjugue | Posterieur |
|---------------|----------------|------------|
| Gaussian (moyenne) | Gaussian | Gaussian |
| Gaussian (precision) | Gamma | Gamma |
| Bernoulli | Beta | Beta |
| Discrete | Dirichlet | Dirichlet |

### Pour notre modele cycliste

- **Moyenne** : prior Gaussian vague $\mu \sim \mathcal{N}(15, 100)$
- **Precision** : prior Gamma $\tau \sim \text{Gamma}(2, 0.5)$

## 4. Modele Simple : Une Gaussienne

In [2]:
// Definition du modele
// Prior sur la moyenne : Gaussian vague centree sur 15 min
Variable<double> dureeMoyenne = Variable.GaussianFromMeanAndPrecision(15, 0.01);  // precision = 0.01 -> variance = 100

// Prior sur la precision (inverse de la variance)
Variable<double> bruitTrafic = Variable.GammaFromShapeAndScale(2, 0.5);

In [3]:
// Definition des temps de trajet observes
Variable<double> dureeLundi = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);
Variable<double> dureeMardi = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);
Variable<double> dureeMercredi = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);

// Observations
dureeLundi.ObservedValue = 13;
dureeMardi.ObservedValue = 17;
dureeMercredi.ObservedValue = 16;

In [4]:
// Inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian moyennePosterieure = moteur.Infer<Gaussian>(dureeMoyenne);
Gamma bruitPosterieur = moteur.Infer<Gamma>(bruitTrafic);

Console.WriteLine($"Moyenne a posteriori : {moyennePosterieure}");
Console.WriteLine($"Precision a posteriori : {bruitPosterieur}");
Console.WriteLine($"\nMoyenne estimee : {moyennePosterieure.GetMean():F2} min");
Console.WriteLine($"Ecart-type du bruit : {Math.Sqrt(1/bruitPosterieur.GetMean()):F2} min");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Moyenne a posteriori : Gaussian(15,33, 1,32)
Precision a posteriori : Gamma(2,242, 0,2445)[mean=0,5482]

Moyenne estimee : 15,33 min
Ecart-type du bruit : 1,35 min


### Analyse des resultats

**Sortie obtenue** :
- `Moyenne a posteriori : Gaussian(15,33, 1,32)` → moyenne ~15.3 min avec precision 1.32
- `Precision a posteriori : Gamma(2,242, 0,2445)` → precision moyenne ~0.55

**Interpretation** :

| Aspect | Valeur | Explication |
|--------|--------|-------------|
| Moyenne | 15.33 | Proche de la moyenne empirique (13+17+16)/3 = 15.33 ✓ |
| Ecart-type bruit | 1.35 | Variabilite typique entre trajets |
| Precision posterior | 1.32 | Plus concentre que le prior (0.01) |

Remarquez le message **"Iterating: ....50"** qui indique que l'algorithme Expectation Propagation a effectue 50 iterations pour converger. Contrairement au modele Bernoulli exact du notebook 1, les modeles Gaussiens avec precision inconnue necessitent une inference **iterative**.

## 5. Prediction du Temps de Demain

In [5]:
// Prediction pour demain
Variable<double> dureeDemain = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);
Gaussian distribDemain = moteur.Infer<Gaussian>(dureeDemain);

Console.WriteLine($"Prediction demain : {distribDemain}");
Console.WriteLine($"Temps moyen estime : {distribDemain.GetMean():F2} min");
Console.WriteLine($"Ecart-type de la prediction : {Math.Sqrt(distribDemain.GetVariance()):F2} min");

// Intervalle de confiance a 95%
double mean = distribDemain.GetMean();
double std = Math.Sqrt(distribDemain.GetVariance());
Console.WriteLine($"\nIntervalle de confiance 95% : [{mean - 1.96*std:F1}, {mean + 1.96*std:F1}] min");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Prediction demain : Gaussian(15,33, 4,613)
Temps moyen estime : 15,33 min
Ecart-type de la prediction : 2,15 min

Intervalle de confiance 95% : [11,1, 19,5] min


### Interpretation de la prediction

**Sortie** : `Gaussian(15,33, 4,613)` avec ecart-type 2.15 min

**Pourquoi l'ecart-type de la prediction (2.15) est plus grand que celui du bruit (1.35) ?**

La prediction combine **deux sources d'incertitude** :

1. **Incertitude epistemique** : Nous ne connaissons pas exactement la vraie moyenne (notre estimation a une variance)
2. **Incertitude aleatoire** : Meme si nous connaissions parfaitement la moyenne, chaque trajet varie autour d'elle

$$\text{Var}(\text{prediction}) = \text{Var}(\mu) + \text{Var}(\text{bruit}) \approx 0.76 + 1.82 = 4.58$$

L'intervalle de confiance 95% `[11.1, 19.5] min` est assez large avec seulement 3 observations. Plus de donnees le reduiront.

## 6. Calcul de Probabilites

In [6]:
// Quelle est la probabilite que le trajet dure moins de 18 minutes ?
Bernoulli probMoinsDe18 = moteur.Infer<Bernoulli>(dureeDemain < 18.0);
Console.WriteLine($"P(trajet < 18 min) = {probMoinsDe18.GetProbTrue():F2}");

// Et moins de 15 minutes ?
Bernoulli probMoinsDe15 = moteur.Infer<Bernoulli>(dureeDemain < 15.0);
Console.WriteLine($"P(trajet < 15 min) = {probMoinsDe15.GetProbTrue():F2}");

// Entre 14 et 18 minutes ?
Variable<bool> entre14et18 = (dureeDemain > 14.0) & (dureeDemain < 18.0);
Bernoulli probEntre = moteur.Infer<Bernoulli>(entre14et18);
Console.WriteLine($"P(14 < trajet < 18 min) = {probEntre.GetProbTrue():F2}");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
P(trajet < 18 min) = 0,89
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
P(trajet < 15 min) = 0,44
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
P(14 < trajet < 18 min) = 0,65


### Interpretation des probabilites

**Resultats obtenus** :
- `P(trajet < 18 min) = 0.89` : Forte probabilite d'arriver en moins de 18 minutes
- `P(trajet < 15 min) = 0.44` : Environ 1 chance sur 2 d'arriver en moins de 15 minutes
- `P(14 < trajet < 18 min) = 0.65` : Probabilite moderee d'etre dans cette fourchette

**Applications pratiques** :

| Question | Probabilite | Decision |
|----------|-------------|----------|
| "Puis-je arriver a l'heure si je pars 18 min avant ?" | 89% | Oui, marge confortable |
| "Et si je pars seulement 15 min avant ?" | 44% | Risque, 1 fois sur 2 en retard |
| "Fenetre optimale de depart ?" | 65% entre 14-18 min | Viser ~16-17 min de marge |

> **Note technique** : Infer.NET permet de calculer ces probabilites directement a partir de la distribution predictive, sans avoir a faire d'integration numerique explicite. La comparaison `dureeDemain < 18.0` cree une nouvelle variable Bernoulli dont la probabilite est inferee automatiquement.

## 7. Restructuration avec Classes

Pour des modeles plus complexes, il est preferable d'encapsuler le code dans des classes reutilisables. Cette architecture permet :

### Avantages de la restructuration

| Avantage | Description |
|----------|-------------|
| **Reutilisabilite** | Les memes classes servent pour differents jeux de donnees |
| **Separation des responsabilites** | Entrainement et prediction dans des classes distinctes |
| **Apprentissage en ligne** | Les posterieurs d'une execution deviennent les priors de la suivante |
| **Maintenabilite** | Le code du modele est isole et testable |

### Architecture proposee

```
DonneesCycliste (struct)
    |-- DistribMoyenne : Gaussian
    |-- DistribBruitTraffic : Gamma

CyclisteBase (classe abstraite)
    |-- MoteurInference
    |-- Moyenne, Bruit (variables)
    |-- MoyenneAPriori, BruitAPriori (priors)
    |-- CreationModeleBayesien()
    |-- DefinirDistributions()
    
    EntrainementCycliste : CyclisteBase
        |-- TempsDeTrajet[] (observations)
        |-- CalculePosterieurs() -> DonneesCycliste
    
    PredictionCycliste : CyclisteBase
        |-- demainTemps (variable a predire)
        |-- EstimerTempsDemain() -> Gaussian
        |-- EstimerTempsDemainInferieurA(seuil) -> Bernoulli
```

### Flux de donnees

1. **Initialisation** : Creer des priors vagues (`DonneesCycliste`)
2. **Entrainement** : `EntrainementCycliste.CalculePosterieurs(observations)` → nouveaux posterieurs
3. **Prediction** : `PredictionCycliste.DefinirDistributions(posterieurs)` puis `EstimerTempsDemain()`
4. **Apprentissage en ligne** : Les posterieurs de l'etape 2 deviennent les priors de l'etape suivante

## 6bis. Gaussienne Tronquée

La **Gaussienne tronquée** est une distribution où les valeurs sont contraintes à un intervalle. C'est utile quand les données ne peuvent pas prendre certaines valeurs (ex: temps de trajet > 0, température entre 0 et 100°C).

### Syntaxe Infer.NET

```csharp
// Gaussienne tronquée entre lower et upper
Variable<double> x = Variable.TruncatedGaussian(mean, variance, lower, upper);

// Gaussienne positive (tronquée à 0)
Variable<double> y = Variable.TruncatedGaussian(10, 4, 0, double.PositiveInfinity);
```

### Exemple : Temps de trajet (toujours positif)

In [7]:
// Exemple de Gaussienne tronquee - cas simple

Console.WriteLine("=== Gaussienne Tronquee ===\n");

// Cas 1 : Prediction avec contrainte de positivite
// On connait la moyenne et precision, on veut predire une valeur positive

Variable<double> moyenneConnue = Variable.Observed(4.0);
Variable<double> precisionConnue = Variable.Observed(1.0);

// Variable aleatoire avec contrainte de positivite
Variable<double> tempsPrediction = Variable.GaussianFromMeanAndPrecision(moyenneConnue, precisionConnue);
Variable.ConstrainPositive(tempsPrediction);

InferenceEngine moteurTronque = new InferenceEngine(new ExpectationPropagation());
moteurTronque.Compiler.CompilerChoice = CompilerChoice.Roslyn;

// Comparer les distributions avec et sans troncature
Console.WriteLine("Distribution Gaussienne standard : N(4, 1)");
Console.WriteLine($"  -> Moyenne = 4.0, Ecart-type = 1.0");
Console.WriteLine($"  -> P(temps < 0) = {Gaussian.FromMeanAndVariance(4, 1).GetProbLessThan(0):F4}");

var distribTronquee = moteurTronque.Infer<Gaussian>(tempsPrediction);
Console.WriteLine($"\nDistribution Gaussienne tronquee [0, +inf) :");
Console.WriteLine($"  -> {distribTronquee}");
Console.WriteLine($"  -> Moyenne ajustee = {distribTronquee.GetMean():F3}");

// Cas 2 : Effet de la troncature proche de zero
Console.WriteLine("\n--- Effet pres de la borne ---");
Variable<double> moyenneProche = Variable.Observed(1.0);  // Proche de 0
Variable<double> precisionProche = Variable.Observed(0.25);  // Variance = 4, ecart-type = 2
Variable<double> tempsProche = Variable.GaussianFromMeanAndPrecision(moyenneProche, precisionProche);
Variable.ConstrainPositive(tempsProche);

var distribProche = moteurTronque.Infer<Gaussian>(tempsProche);
Console.WriteLine($"N(1, 4) standard : P(temps < 0) = {Gaussian.FromMeanAndVariance(1, 4).GetProbLessThan(0):F3} (~31%)");
Console.WriteLine($"N(1, 4) tronquee : {distribProche}");
Console.WriteLine($"  -> Moyenne ajustee de 1.0 a {distribProche.GetMean():F2} (effet de la troncature)");

=== Gaussienne Tronquee ===

Distribution Gaussienne standard : N(4, 1)
  -> Moyenne = 4.0, Ecart-type = 1.0
  -> P(temps < 0) = 0,0000
Compiling model...done.

Distribution Gaussienne tronquee [0, +inf) :
  -> Gaussian(4, 0,9995)
  -> Moyenne ajustee = 4,000

--- Effet pres de la borne ---
Compiling model...done.
N(1, 4) standard : P(temps < 0) = 0,309 (~31%)
N(1, 4) tronquee : Gaussian(2,018, 1,945)
  -> Moyenne ajustee de 1.0 a 2,02 (effet de la troncature)


### Quand utiliser la Gaussienne Tronquée ?

| Situation | Intervalle | Exemple |
|-----------|------------|---------|
| **Quantités positives** | [0, +∞) | Temps, distances, prix |
| **Probabilités** | [0, 1] | Taux de conversion, précision |
| **Températures physiques** | [-273.15, +∞) | Température en Celsius |
| **Notes/Scores** | [0, 20] | Notes d'examen |

**Avantages de la troncature** :

1. **Réalisme** : Les prédictions respectent les contraintes physiques
2. **Meilleure estimation** : La variance n'est pas "gaspillée" sur des valeurs impossibles
3. **Intervalles de confiance valides** : Pas d'aberrations dans les prédictions

**Note technique** : `Variable.ConstrainPositive(x)` est équivalent à `Variable.ConstrainTrue(x > 0)` mais plus efficace car utilise un opérateur spécialisé pour les Gaussiennes tronquées.

In [8]:
// Structure pour stocker les posterieurs
public struct DonneesCycliste
{
    public Gaussian DistribMoyenne;
    public Gamma DistribBruitTraffic;
    
    public DonneesCycliste(Gaussian moyenne, Gamma precision)
    {
        DistribMoyenne = moyenne;
        DistribBruitTraffic = precision;
    }
}

// Classe de base avec les elements communs
public class CyclisteBase
{
    public InferenceEngine MoteurInference;
    protected Variable<double> Moyenne;
    protected Variable<double> Bruit;
    protected Variable<Gaussian> MoyenneAPriori;
    protected Variable<Gamma> BruitAPriori;

    public virtual void CreationModeleBayesien()
    {
        MoyenneAPriori = Variable.New<Gaussian>();
        BruitAPriori = Variable.New<Gamma>();
        Moyenne = Variable.Random<double, Gaussian>(MoyenneAPriori);
        Bruit = Variable.Random<double, Gamma>(BruitAPriori);

        if (MoteurInference == null)
        {
            MoteurInference = new InferenceEngine(new ExpectationPropagation());
            MoteurInference.Compiler.CompilerChoice = CompilerChoice.Roslyn;
        }
    }

    public virtual void DefinirDistributions(DonneesCycliste distribsApriori)
    {
        MoyenneAPriori.ObservedValue = distribsApriori.DistribMoyenne;
        BruitAPriori.ObservedValue = distribsApriori.DistribBruitTraffic;
    }
}

Console.WriteLine("Classe CyclisteBase definie.");

Classe CyclisteBase definie.


In [9]:
// Classe d'entrainement
public class EntrainementCycliste : CyclisteBase
{
    protected VariableArray<double> TempsDeTrajet;
    protected Variable<int> NombreDeTrajets;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        NombreDeTrajets = Variable.New<int>();
        Range indiceTrajet = new Range(NombreDeTrajets);
        TempsDeTrajet = Variable.Array<double>(indiceTrajet);
        using (Variable.ForEach(indiceTrajet))
        {
            TempsDeTrajet[indiceTrajet] = Variable.GaussianFromMeanAndPrecision(Moyenne, Bruit);
        }
    }

    public DonneesCycliste CalculePosterieurs(double[] donneesObservees)
    {
        DonneesCycliste posterieurs;
        NombreDeTrajets.ObservedValue = donneesObservees.Length;
        TempsDeTrajet.ObservedValue = donneesObservees;
        posterieurs.DistribMoyenne = MoteurInference.Infer<Gaussian>(Moyenne);
        posterieurs.DistribBruitTraffic = MoteurInference.Infer<Gamma>(Bruit);
        return posterieurs;
    }
}

// Classe de prediction
public class PredictionCycliste : CyclisteBase
{
    public Variable<double> demainTemps;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        demainTemps = Variable.GaussianFromMeanAndPrecision(Moyenne, Bruit);
    }

    public Gaussian EstimerTempsDemain()
    {
        return MoteurInference.Infer<Gaussian>(demainTemps);
    }

    public Bernoulli EstimerTempsDemainInferieurA(double duree)
    {
        return MoteurInference.Infer<Bernoulli>(demainTemps < duree);
    }
}

Console.WriteLine("Classes EntrainementCycliste et PredictionCycliste definies.");

Classes EntrainementCycliste et PredictionCycliste definies.


In [10]:
// Utilisation des classes
double[] donneesTrajets = new[] { 13.0, 17.0, 20.0, 25.0, 16.0, 11.0, 16.0, 14.0, 12.5 };

// Priors vagues
DonneesCycliste mesDistributions = new DonneesCycliste(
    Gaussian.FromMeanAndPrecision(15, 0.01),  // Prior moyenne
    Gamma.FromShapeAndScale(2, 0.5));         // Prior precision

// Entrainement
EntrainementCycliste entrainement = new EntrainementCycliste();
entrainement.CreationModeleBayesien();
entrainement.DefinirDistributions(mesDistributions);
DonneesCycliste posterieurs = entrainement.CalculePosterieurs(donneesTrajets);

Console.WriteLine($"Moyenne a posteriori : {posterieurs.DistribMoyenne}");
Console.WriteLine($"Precision a posteriori : {posterieurs.DistribBruitTraffic}");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Moyenne a posteriori : Gaussian(16,04, 1,772)
Precision a posteriori : Gamma(5,114, 0,01585)[mean=0,08106]


### Interpretation des resultats d'entrainement

**Donnees utilisees** : 9 observations `{13, 17, 20, 25, 16, 11, 16, 14, 12.5}`

**Resultats** :
- `Moyenne a posteriori : Gaussian(16,04, 1,772)` → moyenne ~16 min avec precision 1.77
- `Precision a posteriori : Gamma(5,114, 0,01585)` → precision moyenne ~0.08

**Comparaison avec le modele simple (3 observations)** :

| Aspect | 3 observations | 9 observations |
|--------|---------------|----------------|
| Moyenne estimee | 15.33 min | 16.04 min |
| Precision du posterior | 1.32 | 1.77 |
| Ecart-type bruit | 1.35 min | 3.51 min |

Avec plus de donnees (dont certaines extremes comme 25 min), le modele :
- Ajuste la moyenne vers le haut (16 vs 15.33)
- **Augmente l'incertitude sur le bruit** car les donnees sont plus dispersees
- Gagne en **confiance sur la moyenne** (precision 1.77 > 1.32)

In [11]:
// Prediction avec les posterieurs
PredictionCycliste prediction = new PredictionCycliste();
prediction.CreationModeleBayesien();
prediction.DefinirDistributions(posterieurs);  // Utiliser les posterieurs comme priors

Gaussian predictionDemain = prediction.EstimerTempsDemain();
Console.WriteLine($"Prediction demain : {predictionDemain}");
Console.WriteLine($"Ecart-type : {Math.Sqrt(predictionDemain.GetVariance()):F2} min");

double probMoins18 = prediction.EstimerTempsDemainInferieurA(18).GetProbTrue();
Console.WriteLine($"P(trajet < 18 min) = {probMoins18:F2}");

Compiling model...done.
Prediction demain : Gaussian(16,04, 17,11)
Ecart-type : 4,14 min
Compiling model...done.
P(trajet < 18 min) = 0,68


### Interpretation de la prediction

**Prediction** : `Gaussian(16,04, 17,11)` avec ecart-type 4.14 min

**Pourquoi l'ecart-type est-il plus grand (4.14 min) qu'avec 3 observations (2.15 min) ?**

Cela peut sembler contre-intuitif : plus de donnees, mais plus d'incertitude ? L'explication :

1. **Donnees plus dispersees** : Les 9 observations incluent des extremes (11 a 25 min)
2. **Meilleure estimation de la variance reelle** : Le modele a appris que les trajets varient beaucoup
3. **Honnetete bayesienne** : Le modele reflète fidelement l'incertitude observee dans les donnees

**Probabilite P(trajet < 18 min) = 0.68** : Avec ces donnees plus realistes, la confiance d'arriver en moins de 18 min est reduite (68% vs 89% avec 3 observations seulement).

## 8. Apprentissage en Ligne

L'apprentissage en ligne permet de mettre a jour le modele incrementalement avec de nouvelles donnees, sans retraiter tout l'historique.

**Principe** : Les posterieurs de la semaine precedente deviennent les priors de la semaine suivante.

In [12]:
// Nouvelle semaine de donnees
double[] semaineSuivante = new double[] { 18, 25, 30, 14, 11 };

// Utiliser les posterieurs comme nouveaux priors
entrainement.DefinirDistributions(posterieurs);
DonneesCycliste posterieursSemaine2 = entrainement.CalculePosterieurs(semaineSuivante);

Console.WriteLine("=== Apres semaine 2 ===");
Console.WriteLine($"Moyenne a posteriori : {posterieursSemaine2.DistribMoyenne}");
Console.WriteLine($"Precision a posteriori : {posterieursSemaine2.DistribBruitTraffic}");

// Nouvelle prediction
prediction.DefinirDistributions(posterieursSemaine2);
Gaussian nouvellePrediction = prediction.EstimerTempsDemain();
Console.WriteLine($"\nNouvelle prediction demain : {nouvellePrediction}");
Console.WriteLine($"Ecart-type : {Math.Sqrt(nouvellePrediction.GetVariance()):F2} min");

Iterating: 
.........|.........|.........|.........|.........| 50
=== Apres semaine 2 ===
Moyenne a posteriori : Gaussian(16,92, 1,436)
Precision a posteriori : Gamma(7,012, 0,005291)[mean=0,0371]

Nouvelle prediction demain : Gaussian(16,92, 32,87)
Ecart-type : 5,73 min


### Analyse de l'apprentissage en ligne

**Nouvelles donnees semaine 2** : `{18, 25, 30, 14, 11}` - incluant des trajets tres longs (25, 30 min)

**Evolution des posterieurs** :

| Parametre | Apres semaine 1 | Apres semaine 2 | Evolution |
|-----------|-----------------|-----------------|-----------|
| Moyenne | 16.04 min | 16.92 min | +0.88 min |
| Precision moyenne | 1.77 | 1.44 | Confiance stable |
| Ecart-type bruit | 3.51 min | 5.73 min | +2.22 min |

**Observations cles** :

1. **Moyenne en hausse** : Les trajets longs (25, 30 min) tirent la moyenne vers le haut
2. **Variance en hausse** : Le modele apprend que les trajets sont encore plus variables que prevu
3. **Accumulation des donnees** : 14 observations totales (9 + 5) sans retraitement

**Avantage de l'apprentissage en ligne** :
- Pas besoin de stocker toutes les observations historiques
- Les posterieurs **resument** toute l'information passee
- Mise a jour incrementale efficace pour les systemes en production

> **Attention** : Si la distribution des donnees change radicalement (ex: nouveau trajet), les anciens posterieurs peuvent freiner l'adaptation. Dans ce cas, "oublier" partiellement le passe peut etre necessaire.

## 9. Probleme : Evenements Extraordinaires

### Observation

Nos donnees contiennent parfois des temps de trajet anormalement longs (25, 30 min) dus a des evenements extraordinaires :
- Accident sur la route
- Meteo extreme
- Travaux

### Solution

Un **modele de melange de Gaussiennes** peut capturer cette bimodalite :
- **Composante 1** : Trajets ordinaires (~15 min)
- **Composante 2** : Trajets extraordinaires (~30 min)

$$p(x) = \pi_1 \cdot \mathcal{N}(x|\mu_1, \tau_1^{-1}) + \pi_2 \cdot \mathcal{N}(x|\mu_2, \tau_2^{-1})$$

ou $\pi_1 + \pi_2 = 1$ sont les poids du melange.

## 10. Modele de Melange de Gaussiennes

In [13]:
// Structure pour le modele mixte
public struct DonneesCyclisteMixte
{
    public Gaussian[] DistribMoyenne;       // Une par composante
    public Gamma[] DistribBruitTraffic;     // Une par composante
    public Dirichlet DistribMixe;           // Poids du melange
}

// Classe de base pour le modele mixte
public class CyclisteBaseMixte
{
    public InferenceEngine MoteurInference;
    protected int NombreComposantes = 2;
    protected VariableArray<Gaussian> MoyennesAPriori;
    protected VariableArray<Gamma> BruitsAPriori;
    protected Variable<Dirichlet> MixeAPriori;
    protected VariableArray<double> Moyennes;
    protected VariableArray<double> Bruits;
    protected Variable<Vector> Mixe;

    public virtual void CreationModeleBayesien()
    {
        Range indiceComposants = new Range(NombreComposantes);
        
        // VMP est recommande pour les melanges
        MoteurInference = new InferenceEngine(new VariationalMessagePassing());
        MoteurInference.Compiler.CompilerChoice = CompilerChoice.Roslyn;
        
        // Priors pour chaque composante
        MoyennesAPriori = Variable.Array<Gaussian>(indiceComposants);
        BruitsAPriori = Variable.Array<Gamma>(indiceComposants);
        Moyennes = Variable.Array<double>(indiceComposants);
        Bruits = Variable.Array<double>(indiceComposants);
        
        using (Variable.ForEach(indiceComposants))
        {
            Moyennes[indiceComposants] = Variable<double>.Random(MoyennesAPriori[indiceComposants]);
            Bruits[indiceComposants] = Variable<double>.Random(BruitsAPriori[indiceComposants]);
        }
        
        // Prior sur les poids du melange (Dirichlet)
        MixeAPriori = Variable.New<Dirichlet>();
        Mixe = Variable<Vector>.Random(MixeAPriori);
        Mixe.SetValueRange(indiceComposants);
    }

    public virtual void DefinirDistributions(DonneesCyclisteMixte distribsApriori)
    {
        MoyennesAPriori.ObservedValue = distribsApriori.DistribMoyenne;
        BruitsAPriori.ObservedValue = distribsApriori.DistribBruitTraffic;
        MixeAPriori.ObservedValue = distribsApriori.DistribMixe;
    }
}

Console.WriteLine("Classe CyclisteBaseMixte definie.");

Classe CyclisteBaseMixte definie.



(6,22): warning CS0649: Le champ 'DonneesCyclisteMixte.DistribMixe' n'est jamais assigné et aura toujours sa valeur par défaut null

(5,20): warning CS0649: Le champ 'DonneesCyclisteMixte.DistribBruitTraffic' n'est jamais assigné et aura toujours sa valeur par défaut null

(4,23): warning CS0649: Le champ 'DonneesCyclisteMixte.DistribMoyenne' n'est jamais assigné et aura toujours sa valeur par défaut null



In [14]:
// Classe d'entrainement pour le modele mixte
public class EntrainementCyclisteMixte : CyclisteBaseMixte
{
    protected Variable<int> NombreDeTrajets;
    protected VariableArray<double> TempsDeTrajet;
    protected VariableArray<int> ComposantesTrajets;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        NombreDeTrajets = Variable.New<int>();
        Range indiceTrajet = new Range(NombreDeTrajets);
        TempsDeTrajet = Variable.Array<double>(indiceTrajet);
        ComposantesTrajets = Variable.Array<int>(indiceTrajet);
        
        using (Variable.ForEach(indiceTrajet))
        {
            // Selection de la composante selon les poids du melange
            ComposantesTrajets[indiceTrajet] = Variable.Discrete(Mixe);
            
            // Variable.Switch selectionne la composante appropriee
            using (Variable.Switch(ComposantesTrajets[indiceTrajet]))
            {
                TempsDeTrajet[indiceTrajet].SetTo(
                    Variable.GaussianFromMeanAndPrecision(
                        Moyennes[ComposantesTrajets[indiceTrajet]], 
                        Bruits[ComposantesTrajets[indiceTrajet]]));
            }
        }
    }

    public DonneesCyclisteMixte CalculePosterieurs(double[] donneesObservees)
    {
        DonneesCyclisteMixte posterieurs;
        NombreDeTrajets.ObservedValue = donneesObservees.Length;
        TempsDeTrajet.ObservedValue = donneesObservees;
        posterieurs.DistribMoyenne = MoteurInference.Infer<Gaussian[]>(Moyennes);
        posterieurs.DistribBruitTraffic = MoteurInference.Infer<Gamma[]>(Bruits);
        posterieurs.DistribMixe = MoteurInference.Infer<Dirichlet>(Mixe);
        return posterieurs;
    }
}

Console.WriteLine("Classe EntrainementCyclisteMixte definie.");

Classe EntrainementCyclisteMixte definie.


In [15]:
// Classe de prediction pour le modele mixte
public class PredictionCyclisteMixte : CyclisteBaseMixte
{
    public Variable<double> demainTemps;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        Variable<int> indiceComposant = Variable.Discrete(Mixe);
        demainTemps = Variable.New<double>();
        using (Variable.Switch(indiceComposant))
        {
            demainTemps.SetTo(Variable.GaussianFromMeanAndPrecision(
                Moyennes[indiceComposant], Bruits[indiceComposant]));
        }
    }

    public Gaussian EstimerTempsDemain()
    {
        return MoteurInference.Infer<Gaussian>(demainTemps);
    }
}

Console.WriteLine("Classe PredictionCyclisteMixte definie.");

Classe PredictionCyclisteMixte definie.


## 11. Entrainement du Modele Mixte

In [16]:
// Donnees avec evenements extraordinaires
double[] donneesMixtes = new[] { 13.0, 17.0, 20.0, 25.0, 16.0, 11.0, 16.0, 25.0, 12.5, 30.0 };

// Priors pour le modele mixte
DonneesCyclisteMixte priorsMMixtes = new DonneesCyclisteMixte
{
    DistribMoyenne = new Gaussian[]
    {
        new Gaussian(15, 100),  // Composante 1 : Ordinaire (~15 min)
        new Gaussian(30, 100)   // Composante 2 : Extraordinaire (~30 min)
    },
    DistribBruitTraffic = new Gamma[]
    {
        Gamma.FromShapeAndScale(2, 0.5),
        Gamma.FromShapeAndScale(2, 0.5)
    },
    DistribMixe = new Dirichlet(1, 1)  // Prior uniforme sur les poids
};

// Entrainement
EntrainementCyclisteMixte entrainementMixte = new EntrainementCyclisteMixte();
entrainementMixte.CreationModeleBayesien();
entrainementMixte.DefinirDistributions(priorsMMixtes);
DonneesCyclisteMixte posterieursMixte = entrainementMixte.CalculePosterieurs(donneesMixtes);

Console.WriteLine("=== Resultats du modele de melange ===");
Console.WriteLine($"\nComposante 1 (Ordinaire):");
Console.WriteLine($"  Moyenne : {posterieursMixte.DistribMoyenne[0]}");
Console.WriteLine($"  Precision : {posterieursMixte.DistribBruitTraffic[0]}");

Console.WriteLine($"\nComposante 2 (Extraordinaire):");
Console.WriteLine($"  Moyenne : {posterieursMixte.DistribMoyenne[1]}");
Console.WriteLine($"  Precision : {posterieursMixte.DistribBruitTraffic[1]}");

Console.WriteLine($"\nPoids du melange : {posterieursMixte.DistribMixe}");
var poidsMoyens = posterieursMixte.DistribMixe.GetMean();
Console.WriteLine($"  -> P(ordinaire) = {poidsMoyens[0]:F2}, P(extraordinaire) = {poidsMoyens[1]:F2}");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Resultats du modele de melange ===

Composante 1 (Ordinaire):
  Moyenne : Gaussian(15,07, 0,8674)
  Precision : Gamma(5,498, 0,02972)[mean=0,1634]

Composante 2 (Extraordinaire):
  Moyenne : Gaussian(26,69, 1,146)
  Precision : Gamma(3,502, 0,08199)[mean=0,2871]

Poids du melange : Dirichlet(7,995 4,005)
  -> P(ordinaire) = 0,67, P(extraordinaire) = 0,33


### Analyse du modele de melange

**Resultats obtenus** :

| Composante | Moyenne | Interpretation |
|------------|---------|----------------|
| Ordinaire | 15.07 min | Trajets normaux : {13, 17, 16, 11, 12.5} ✓ |
| Extraordinaire | 26.69 min | Trajets longs : {20, 25, 25, 30} ✓ |

**Poids du melange** : `Dirichlet(7.995, 4.005)` → P(ordinaire) = 67%, P(extraordinaire) = 33%

Avec 10 observations, le modele a correctement identifie :
- **~7 trajets ordinaires** (prior 1 + ~6 observations assignees)
- **~4 trajets extraordinaires** (prior 1 + ~3 observations assignees)

> **Note sur VMP** : Le modele utilise Variational Message Passing (`VariationalMessagePassing`) au lieu d'Expectation Propagation. VMP est recommande pour les modeles de melange car il gere mieux les variables latentes discretes (l'assignation aux composantes).

In [17]:
// Prediction avec le modele mixte
PredictionCyclisteMixte predictionMixte = new PredictionCyclisteMixte();
predictionMixte.CreationModeleBayesien();
predictionMixte.DefinirDistributions(posterieursMixte);

Gaussian predictionMixteDemain = predictionMixte.EstimerTempsDemain();
Console.WriteLine($"\nPrediction demain (modele mixte) : {predictionMixteDemain}");
Console.WriteLine($"Ecart-type : {Math.Sqrt(predictionMixteDemain.GetVariance()):F2} min");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50

Prediction demain (modele mixte) : Gaussian(18,48, 33,39)
Ecart-type : 5,78 min


## 12. Variable.Switch Explique

### Syntaxe

`Variable.Switch` permet de selectionner dynamiquement une valeur parmi plusieurs options basees sur un indice discret.

```csharp
Variable<int> indice = Variable.Discrete(poids);  // Selectionne 0, 1, 2...

using (Variable.Switch(indice))
{
    // indice est utilise pour selectionner parmi des tableaux
    resultat.SetTo(valeurs[indice]);
}
```

### Applications

- **Modeles de melange** : Selection de la composante
- **Modeles hierarchiques** : Selection du groupe
- **Hidden Markov Models** : Selection de l'etat cache

## 13. Exercice : Melange a 3 Composantes

### Enonce

Modifiez le modele de melange pour utiliser **3 composantes** :
1. Trajets rapides (~10 min)
2. Trajets normaux (~18 min)
3. Trajets longs (~30 min)

### Donnees

```csharp
double[] donnees3Comp = new[] { 8, 10, 12, 18, 17, 19, 20, 28, 32, 35, 11, 18, 30 };
```

### Indice

- Changez `NombreComposantes = 3` dans la classe de base
- Ajustez les priors pour 3 composantes
- Utilisez `Dirichlet(1, 1, 1)` pour les poids

In [18]:
// EXERCICE : Implementez un melange a 3 composantes

// Modifiez la classe de base pour 3 composantes
public class CyclisteBase3Composantes : CyclisteBaseMixte
{
    public CyclisteBase3Composantes()
    {
        NombreComposantes = 3;  // <- Modification ici
    }
}

public class Entrainement3Composantes : CyclisteBase3Composantes
{
    protected Variable<int> NombreDeTrajets;
    protected VariableArray<double> TempsDeTrajet;
    protected VariableArray<int> ComposantesTrajets;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        NombreDeTrajets = Variable.New<int>();
        Range indiceTrajet = new Range(NombreDeTrajets);
        TempsDeTrajet = Variable.Array<double>(indiceTrajet);
        ComposantesTrajets = Variable.Array<int>(indiceTrajet);
        
        using (Variable.ForEach(indiceTrajet))
        {
            ComposantesTrajets[indiceTrajet] = Variable.Discrete(Mixe);
            using (Variable.Switch(ComposantesTrajets[indiceTrajet]))
            {
                TempsDeTrajet[indiceTrajet].SetTo(
                    Variable.GaussianFromMeanAndPrecision(
                        Moyennes[ComposantesTrajets[indiceTrajet]], 
                        Bruits[ComposantesTrajets[indiceTrajet]]));
            }
        }
    }

    public DonneesCyclisteMixte CalculePosterieurs(double[] donneesObservees)
    {
        DonneesCyclisteMixte posterieurs;
        NombreDeTrajets.ObservedValue = donneesObservees.Length;
        TempsDeTrajet.ObservedValue = donneesObservees;
        posterieurs.DistribMoyenne = MoteurInference.Infer<Gaussian[]>(Moyennes);
        posterieurs.DistribBruitTraffic = MoteurInference.Infer<Gamma[]>(Bruits);
        posterieurs.DistribMixe = MoteurInference.Infer<Dirichlet>(Mixe);
        return posterieurs;
    }
}

// Donnees
double[] donnees3Comp = new[] { 8.0, 10.0, 12.0, 18.0, 17.0, 19.0, 20.0, 28.0, 32.0, 35.0, 11.0, 18.0, 30.0 };

// Priors pour 3 composantes
DonneesCyclisteMixte priors3Comp = new DonneesCyclisteMixte
{
    DistribMoyenne = new Gaussian[]
    {
        new Gaussian(10, 100),  // Rapide
        new Gaussian(18, 100),  // Normal
        new Gaussian(30, 100)   // Long
    },
    DistribBruitTraffic = new Gamma[]
    {
        Gamma.FromShapeAndScale(2, 0.5),
        Gamma.FromShapeAndScale(2, 0.5),
        Gamma.FromShapeAndScale(2, 0.5)
    },
    DistribMixe = new Dirichlet(1, 1, 1)  // Uniforme sur 3 composantes
};

// Entrainement
Entrainement3Composantes ent3 = new Entrainement3Composantes();
ent3.CreationModeleBayesien();
ent3.DefinirDistributions(priors3Comp);
DonneesCyclisteMixte post3 = ent3.CalculePosterieurs(donnees3Comp);

Console.WriteLine("=== Modele a 3 composantes ===");
for (int i = 0; i < 3; i++)
{
    Console.WriteLine($"\nComposante {i+1}: Moyenne = {post3.DistribMoyenne[i].GetMean():F1} min");
}
Console.WriteLine($"\nPoids : {post3.DistribMixe.GetMean()}");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Modele a 3 composantes ===

Composante 1: Moyenne = 10,2 min

Composante 2: Moyenne = 18,4 min

Composante 3: Moyenne = 31,2 min

Poids : 0,3125 0,375 0,3125


### Analyse de l'exercice a 3 composantes

**Resultats** :
- Composante 1 : ~10.2 min → Trajets rapides {8, 10, 11, 12}
- Composante 2 : ~18.4 min → Trajets normaux {17, 18, 18, 19, 20}
- Composante 3 : ~31.2 min → Trajets longs {28, 30, 32, 35}

**Poids** : `(0.31, 0.38, 0.31)` - Distribution relativement equilibree

Le modele a correctement **segmente** les 13 observations en 3 groupes distincts. Chaque composante a capte un mode de la distribution multimodale des donnees.

> **Attention au label switching** : Dans les modeles de melange, les composantes peuvent s'echanger lors de differentes executions. Ici, Composante 1 est "rapide" mais ce n'est pas garanti a chaque execution. Les priors informatifs (10, 18, 30) aident a ancrer les composantes.

## 14. Resume

| Concept | Description |
|---------|-------------|
| **Gaussian** | Distribution continue pour les valeurs reelles |
| **Gamma** | Prior conjugue pour la precision d'une Gaussienne |
| **Dirichlet** | Prior conjugue pour les poids d'un melange |
| **Variable.Switch** | Selection dynamique parmi plusieurs options |
| **Apprentissage en ligne** | Posterieurs -> Priors pour nouvelles donnees |
| **Modele de melange** | Capture les distributions multimodales |

---

## Prochaine etape

Dans [Infer-3-Factor-Graphs](Infer-3-Factor-Graphs.ipynb), nous explorerons :

- Les graphes de facteurs et leur representation
- L'inference discrete avec le probleme du "Murder Mystery"
- Le paradoxe de Monty Hall
- Le theoreme de Bayes illustre